# Algorithm Scalability

This notebook showcases how `qlbm` enables the analysis of the properties of different QBM algorithms.

We consider two lenses of scalability. First, we analyze the high-level scaling of algorithms as a function of the underlying lattice. Second, we consider the practical scalability of the algorithms after transpilation to more restrictive gate sets. In the latter, we also analyze the performance of compilation software on specific QBM instances.

In [ ]:
import logging.config
from logging import Logger, getLogger
from typing import List

from pytket.extensions.qiskit import AerStateBackend as TketQiskitBackend
from qiskit_aer import AerSimulator

from qlbm.components import (
    CQLBM,
)
from qlbm.infra import CircuitCompiler
from qlbm.lattice import CollisionlessLattice
from qlbm.tools.utils import create_directory_and_parents, get_circuit_properties

In [ ]:
# Import analysis and plotting utilities
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme()

## Step 1: Setup

Before analyzing the scalability of the quantum circuits, we first have to choose some the systems they model and the compilation tools we use. In this case, we vary the number of solid objcets in the fluid domain and study the depth and the number of gates of the resulting CQBM circuit.

In [ ]:
def benchmark(
    lattice_dicts,
    logger: Logger,
    dummy_logger: Logger,
    compiler_platform: List[str],
    target_platform: List[str],
    optimization_levels: List[int],
    backend: TketQiskitBackend | None,
    num_repetitions: int = 5,
) -> None:
    for rep in range(num_repetitions):
        logger.info(f"Repetition #{rep + 1} of {num_repetitions}")
        for count, lattice_dict in enumerate(lattice_dicts):
            for opt_count, optimization_level in enumerate(optimization_levels):
                logger.info(
                    f"Combination #{(count * len(optimization_levels)) + opt_count + 1} of {len(lattice_dicts)*len(optimization_levels)}"
                )

                lattice = CollisionlessLattice(lattice_dict, logger=dummy_logger)

                logger.info(
                    f"Lattice: {lattice.logger_name()}; compiler={compiler_platform}; opt={optimization_level}; num_qubits={lattice.num_total_qubits};"
                )

                algorithm = CQLBM(lattice, logger=dummy_logger)
                logger.info(
                    f"Original circuit has properties: {get_circuit_properties(algorithm.circuit)}"
                )
                compiler = CircuitCompiler(
                    compiler_platform, target_platform, logger=logger
                )

                compiler.compile(
                    compile_object=algorithm,
                    backend=backend,
                    optimization_level=optimization_level,
                )

In [ ]:
ROOT_OUTPUT_DIR = "qlbm-output/benchmark-algorithm-scalability"

create_directory_and_parents(ROOT_OUTPUT_DIR)

In [ ]:
!mkdir -p qlbm-output/benchmark-algorithm-scalability && touch qlbm-output/benchmark-algorithm-scalability/qlbm.log
!:> qlbm-output/benchmark-algorithm-scalability/qlbm.log

In [ ]:
lattices = [
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [],
    },  # 0 Obstacles
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [{"shape": "cuboid", "x": [5, 6], "y": [1, 2], "boundary": "specular"}],
    },  # 1 Obstacle
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [
            {"shape": "cuboid", "x": [5, 6], "y": [1, 2], "boundary": "specular"},
            {"shape": "cuboid", "x": [5, 6], "y": [5, 6], "boundary": "specular"},
        ],
    },  # 2 Obstacles
]

dummy_logger = getLogger("dummy")
# By logging at this point we ignore the output of circuit creation
logging.config.fileConfig("algorithm_scalability_logging.conf")
logger = getLogger("qlbm")

## Step 2: Benchmarking

Now that we have setup our simulations and configered logging, we can simply run the simulations by calling the `benchmark` function!

> **_CAUTION:_** Running the cells below will probably take a few minutes. Each cell should be run exactly once.

In [ ]:
# Compile to Qulacs gate set using Qiskit
logger.info("Session: QISKIT")
benchmark(
    lattices,
    logger,
    dummy_logger,
    "QISKIT",
    "QISKIT",
    [0],
    AerSimulator(),
    num_repetitions=1,
)

In [ ]:
# Compile to Qulacs gate set using Tket
logger.info("Session: TKET")
benchmark(
    lattices,
    logger,
    dummy_logger,
    "TKET",
    "QISKIT",
    [0],
    AerSimulator(),
    num_repetitions=1,
)


## Step 3: Analysis

Once the compilation benchmarks have concluded, the performance logs created by `qlbm` will be under `qlbm-output/benchmark-algorithm-scalability/qlbm.log`.
The scripts below will parse this file, extract useful information, format it, and plot it for convenient analysis.

In [ ]:
log_file = "qlbm-output/benchmark-algorithm-scalability/qlbm.log"
with open(log_file, "r") as f:
    lines = f.readlines()

session_line = [c for c, line in enumerate(lines) if "Session" in line][1]

lines_qiskit = lines[:session_line]
lines_tket = lines[session_line:]

In [ ]:
combination_lines_indices_qiskit, combination_lines_indices_tket = (
    [c for c, line in enumerate(lines_qiskit) if "Combination #" in line],
    [c for c, line in enumerate(lines_tket) if "Combination #" in line],
)
print(len(combination_lines_indices_qiskit + combination_lines_indices_tket))
sections = []
for c in range(len(combination_lines_indices_qiskit + combination_lines_indices_tket)):
    if c < len(combination_lines_indices_qiskit):
        if c < len(combination_lines_indices_qiskit) - 1:
            sections.append(
                lines_qiskit[
                    combination_lines_indices_qiskit[
                        c
                    ] : combination_lines_indices_qiskit[c + 1]
                ]
            )
        else:
            sections.append(lines_qiskit[combination_lines_indices_qiskit[c] :])
    else:
        c_tket = c - len(combination_lines_indices_qiskit)
        if c_tket < len(combination_lines_indices_tket) - 1:
            sections.append(
                lines_tket[
                    combination_lines_indices_tket[
                        c_tket
                    ] : combination_lines_indices_tket[c_tket + 1]
                ]
            )
        else:
            sections.append(lines_tket[combination_lines_indices_tket[c_tket] :])

records = []
for c, section in enumerate(sections):
    section_info = (
        [line for line in section if "Lattice: " in line][0]
        .rstrip("\n")
        .split(": ")[-1]
        .split("; ")
    )
    lattice_name = section_info[0]
    compiler_platform = section_info[1].split("=")[-1]
    opt_level = section_info[2].split("=")[-1]
    num_qubits = section_info[3].split("=")[-1][:-1]

    original_props = (
        [line for line in section if "Original circuit" in line][0]
        .split("(")[-1]
        .rstrip("\n)")
        .split(", ")
    )
    compiled_props = (
        [line for line in section if "Compiled circuit" in line][0]
        .split("(")[-1]
        .rstrip("\n)")
        .split(", ")
    )
    duration = int(
        [line for line in section if "Compilation took" in line][0].split()[-2]
    )

    records.append(
        {
            "Lattice": lattice_name,
            "Dimensions": lattice_name.split("-")[1],
            "Obstacles": int(lattice_name.split("-")[2]),
            "Circuit Qubits": num_qubits,
            "Initial Depth": int(original_props[2]),
            "Initial Gate No.": int(original_props[3]),
            "Compiled Depth": int(compiled_props[2]),
            "Compiled Gate No.": int(compiled_props[3]),
            "Duration (ns)": int(duration),
            "Duration (s)": int(duration) / 1e9,
            "Compiler": "QISKIT"
            if c < len(combination_lines_indices_qiskit)
            else "TKET",
        }
    )

df = pd.DataFrame.from_records(records)

In [ ]:
# We can have a look at the raw data
df

In [ ]:
sns.lineplot(
    df,
    x="Obstacles",
    y="Initial Depth",
    markers=True,
)
plt.xticks(pd.unique(df["Obstacles"]))
plt.title("Circuit Depth Comparison")

In [ ]:
sns.lineplot(
    df,
    x="Obstacles",
    y="Initial Gate No.",
    markers=True,
)
plt.xticks(pd.unique(df["Obstacles"]))
plt.title("Circuit Gate Number Comparison")

In [ ]:
sns.lineplot(
    df,
    x="Obstacles",
    y="Compiled Depth",
    hue="Compiler",
    style="Compiler",
    markers=True,
)
plt.xticks(pd.unique(df["Obstacles"]))
plt.title("Compilation Depth Comparison")

In [ ]:
sns.lineplot(
    df,
    x="Obstacles",
    y="Compiled Gate No.",
    hue="Compiler",
    style="Compiler",
    markers=True,
)
plt.xticks(pd.unique(df["Obstacles"]))
plt.title("Compilation Gate Comparison")

In [ ]:
g = sns.lineplot(
    df,
    x="Obstacles",
    y="Duration (s)",
    hue="Compiler",
    style="Compiler",
    markers=True,
)
plt.xticks(pd.unique(df["Obstacles"]))
plt.title("Compilation Duration Comparison")